## Logging on

Use Selenium to visit https://webapps1.chicago.gov/buildingrecords/ and accept the agreement.

> Think about when you use `.find_element...` and when you use `.find_elementSSS...`

In [469]:
import pandas as pd

import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/Users/jessieblaeser/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache
/var/folders/qx/nkh2pq6x3_193757ddxvrcf40000gp/T/ipykernel_5169/979343879.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [470]:
driver.get('https://webapps1.chicago.gov/buildingrecords/')

In [471]:
driver.find_element(By.ID, 'rbnAgreement1').click()

In [472]:
driver.find_element(By.ID, 'submit').click()

## Searching

Search for **400 E 41ST ST**.

In [473]:
#fullAddress
#driver.find_element(By.ID, "pht_lnm").send_keys("Nguyen")

In [474]:
driver.find_element(By.ID, 'fullAddress').send_keys("400 E 41ST ST.")

In [475]:
driver.find_element(By.ID, 'submit').click()

## Saving tables with pandas

Use pandas to save a CSV of all **permits** to `Permits - 400 E 41ST ST.csv`. Note that there are **different sections of the page**, not just one long permits table.

> - *Tip: When using `.read_html`, try using `flavor='lxml'` and comparing the results to `flavor='html5lib'`. Which works better?*
> - *Tip: You might need to install `html5lib` using `pip`. If so, you'll need to restart the notebook using **Kernel > Restart** before it will work.*

In [476]:
#!pip install html5lib

In [477]:
# table = driver.find_element(By.ID, 'resultstable_permits')
# df = pd.read_html(driver.page_source, flavor='lxml')
#^this returns a list of df's
# df = df[0]
# # QUESTION: i am entirely confused as to how this works!! 

# df.to_csv('Permits - 400 E 41ST ST.csv', index=False)

# df_new = pd.read_csv("Permits - 400 E 41ST ST.csv")
# df_new.head()

#NOTE: lxml replaced my date column with NaN's!!! 

table = driver.find_element(By.ID, 'resultstable_permits')
df = pd.read_html(driver.page_source, flavor='html5lib')
df = df[0]
df.to_csv('Permits - 400 E 41ST ST.csv', index=False)
# df_new = pd.read_csv("Permits - 400 E 41ST ST.csv")
# df_new.head()

#NOTE: so i like html5lib better!!

In [478]:
#find all permiits
# in each tr (row), find the first td (first cell, which is the permit number)

In [479]:
#NOTE: I did it this way first, but I'm seeing now that I did not actually need to?? Sad. 

table = driver.find_element(By.ID, 'resultstable_permits')
rows = table.find_elements(By.TAG_NAME, 'tr')

permit_table = []
for row in rows[1:]: 
    permit_dict = {}
    permit_dict['permit_num'] = row.find_element(By.TAG_NAME, "td").text
    cells = row.find_elements(By.TAG_NAME, 'td')
    #print(cells[1].text)
    permit_dict['date_issued'] = cells[1].text
    #print(cells[2].text)
    permit_dict['work_descrip'] = cells[2].text
    permit_table.append(permit_dict)
permit_table
#len(permit_table)


[{'permit_num': '100845718',
  'date_issued': '10/11/2019',
  'work_descrip': 'ERECT TWO SCAFFOLDS FROM 10/14/2019 TO 10/14/2020.'},
 {'permit_num': '100778302',
  'date_issued': '08/17/2018',
  'work_descrip': "PERMIT EXPIRES ON 10/17/2018 Erection Starts: 8/24/2018, Erection Ends: 8/25/2018. TENT- 1 DAY EVENT 8/25/18 ERECT ONE (1) 40'X60' POLE TENT SETUP 8/24/18- REMOVE 8/25/18 ASSEMBLY **** CONDITIONAL APPROVAL **** SUBJECT TO FIELD INSPECTION **** AOR CERTIFICATE OF INSPECTION REQUIRED ****"},
 {'permit_num': '100721255',
  'date_issued': '08/24/2017',
  'work_descrip': 'PERMIT EXPIRES ON 10/24/2017 ERECTION STARTS: 8/25/2017, ERECTION ENDS: 8/26/2017. SELF CERTIFICATION. REINSTATEMENT OF 2015 PERMIT 100610771 FOR THE PAUL G STEWART CENTER EVENT. SAME 40X60 CANOPY. AAA RENTAL SYSTEM.** CONDITIONAL APPROVAL **SUBJECT TO FIELD INSPECTION * AOR CERTIFICATION OF INSPECTION REQUIRED.'},
 {'permit_num': '100693399',
  'date_issued': '03/03/2017',
  'work_descrip': 'INSTALLATION OF LOW VO

## Saving tables the long way

Save a CSV of all DOB inspections to `Inspections - 400 E 41ST ST.csv`.

This is more complicated than the last one becuse **we also need to save the URL to the inspection** (see how the inspection number is a link?). As a result, you won't be able to use pandas! Instead, you'll need to use a loop and create a list of dictionaries.

You can use Selenium or you can feed the source to BeautifulSoup. You should have approximately 160 rows.

You'll probably need to find the table first, then the rows inside, then the cells inside of each row. You'll probably use lots of list indexing. I might recommend XPath for finding the table.

*Tip: If you get a "list index out of range" error, it's probably due to an issue involving `thead` vs `tbody` elements. What are they? What are they for? What's in them? There are a few ways to troubleshoot it.*

In [480]:
dob_table = driver.find_element(By.ID, 'resultstable_inspections')
# from here, each tr tag is a row
rows = dob_table.find_elements(By.TAG_NAME, 'tr')
#rows[0] = header
# td[0] --> insp #
# -----> td[0] --> a --> href
# td[1] --> insp date
# td[2] --> status
# td[3] --> type descrip

In [481]:
dob_table = driver.find_element(By.ID, 'resultstable_inspections')
# from here, each tr tag is a row
rows = dob_table.find_elements(By.TAG_NAME, 'tr')

dob_list = []
for row in rows[1:]: 
    dob_dict = {}
    cells = row.find_elements(By.TAG_NAME, 'td')
#     print(cells[0].text)
#     print(cells[1].text)
#     print(cells[2].text)
#     print(cells[3].text)
#     print('------------')
    dob_dict['insp_num'] = cells[0].text
    dob_dict['href'] = cells[0].find_element(By.TAG_NAME, 'a').get_attribute('href')
    dob_dict['insp_date'] = cells[1].text
    dob_dict['status'] = cells[2].text
    dob_dict['type_descrip'] = cells[3].text
    dob_list.append(dob_dict)
dob_list
len(dob_list)

160

In [482]:
df = pd.DataFrame(dob_list)
df
df.to_csv('Inspections - 400 E 41ST ST.csv', index=False)

In [483]:
# for row in rows[1:]: 
#     dob_dict = {}
#     cells = row.find_elements(By.TAG_NAME, 'td')
#     print(cells[0].find_element(By.TAG_NAME, 'a').get_attribute('href'))

### Bonus preview of Wednesday's content

**You prrrrrobably shouldn't do this one unless you want a real challenge.**

If you click the inspection number, it'll open up a new window that shows you details of the violations from that visit. Count the number of violations for each visit and save it in a new column called **num_violations**.

Save this file as `Inspections - 400 E 41ST ST - with counts.csv`.

Since clicking the link opens in a new window, we have to say "Hey Selenium, pay attention to that new window!" We do that with `driver.switch_to.window(driver.window_handles[-1])` (each window gets a `window_handle`, and we're just asking the driver to switch to the last one.). A rough sketch of what your code will look like is here:

```python
# Click the link that opens the new window

# Switch to the new window/tab
driver.switch_to.window(driver.window_handles[-1])

# Do your scraping in here

# Close the new window/tab
driver.close()

# Switch back to the original window/tab
driver.switch_to.window(driver.window_handles[0])
```

You'll want to play around with them individually before you try it with the whole set - the ones that pass are very different pages than the ones with violations! There are a few ways to get the number of violations, some easier than others.

In [484]:
# dob_table = driver.find_element(By.ID, 'resultstable_inspections')
# rows = dob_table.find_elements(By.TAG_NAME, 'tr')
# first_cell = rows[1]
# first_cell.find_element(By.TAG_NAME, 'a').click()
# driver.switch_to.window(driver.window_handles[-1])


In [485]:
# table = driver.find_element(By.ID, 'resultstable')
# rows = table.find_elements(By.TAG_NAME, 'tr')
# len(rows)
# #seems first two 'tr' tags are always related to the header of the table, so i'm going to count 
# count = len(rows) - 2
# count

In [486]:
# driver.close()
# driver.switch_to.window(driver.window_handles[0])


In [487]:
# Checking on a similar cell 
dob_table = driver.find_element(By.ID, 'resultstable_inspections')
rows = dob_table.find_elements(By.TAG_NAME, 'tr')
second_cell = rows[2]
second_cell.find_element(By.TAG_NAME, 'a').click()
driver.switch_to.window(driver.window_handles[-1])


In [488]:
table = driver.find_element(By.ID, 'resultstable')
rows = table.find_elements(By.TAG_NAME, 'tr')
# len(rows)
# #seems first two 'tr' tags are always related to the header of the table, so i'm going to count 
count2 = len(rows) - 2
count2

4

In [489]:
driver.close()
driver.switch_to.window(driver.window_handles[0])

In [490]:
# Checking on a 'Passed' cell

In [491]:
dob_table = driver.find_element(By.ID, 'resultstable_inspections')
rows = dob_table.find_elements(By.TAG_NAME, 'tr')
second_cell = rows[6]
second_cell.find_element(By.TAG_NAME, 'a').click()
driver.switch_to.window(driver.window_handles[-1])

In [492]:
# I guess here i'd need to create an if statement or a try/except clause
passed_result = driver.find_element(By.TAG_NAME, 'h5')
passed_result.text

#so if passed_result is not none, then the count = 0 

'There are no Alleged Code Violation records for Inspection 12770690'

In [493]:
#passed_result.text

In [494]:
driver.close()
driver.switch_to.window(driver.window_handles[0])

In [495]:
#I applied everything above here to make sure it worked on its own before 
    # attempting to incorporate into the larger dictionary 

# violation_list = []
# for row in rows[1:10]:
#     violation_dict = {}
#     row.find_element(By.TAG_NAME, 'a').click()
#     driver.switch_to.window(driver.window_handles[-1])
#     try: 
#         table2 = driver.find_element(By.ID, 'resultstable')
#         rows2 = table2.find_elements(By.TAG_NAME, 'tr')
#         count = len(rows2) - 2
#         violation_dict['num_violations'] = count
#     except: 
#         violation_dict['num_violations'] = 0
#     driver.close()
#     driver.switch_to.window(driver.window_handles[0])
#     violation_list.append(violation_dict)
# #violation_list

# # it worksssssssssssss!!!!!!!! 

In [496]:
dob_table = driver.find_element(By.ID, 'resultstable_inspections')
# from here, each tr tag is a row
rows = dob_table.find_elements(By.TAG_NAME, 'tr')
# from there, each td tag is a cell 

dob_list_updated = []
for row in rows[1:]: 
    dob_dict = {}
    cells = row.find_elements(By.TAG_NAME, 'td')
    dob_dict['insp_num'] = cells[0].text
    dob_dict['href'] = cells[0].find_element(By.TAG_NAME, 'a').get_attribute('href')
    dob_dict['insp_date'] = cells[1].text
    dob_dict['status'] = cells[2].text
    dob_dict['type_descrip'] = cells[3].text
    row.find_element(By.TAG_NAME, 'a').click()
    driver.switch_to.window(driver.window_handles[-1])
    try: 
        table2 = driver.find_element(By.ID, 'resultstable')
        rows2 = table2.find_elements(By.TAG_NAME, 'tr')
        count = len(rows2) - 2
        dob_dict['num_violations'] = count
    except: 
        dob_dict['num_violations'] = 0
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    dob_list_updated.append(dob_dict)
len(dob_list_updated)

# :')


160

In [499]:
df = pd.DataFrame(dob_list_updated)
#df.head(10)
df.to_csv('Inspections - 400 E 41ST ST - with counts.csv', index=False)

In [ ]:
#yaaaaaay!!! 